<a href="https://colab.research.google.com/github/ruthsvandewater/z2_matter_qc/blob/RuthFlorianColab/RVEdits_03_18_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulation of $Z_2$ gauge theory including readout-error mitigation

## Import modules

In [ ]:
#Standard modules
import numpy as np
from numpy import pi,sum
import matplotlib.pyplot as plt

#Qiskit
from qiskit import IBMQ, QuantumCircuit, Aer
from qiskit.providers.ibmq import RunnerResult
from qiskit.providers.aer import noise, AerSimulator

#unused imports RV 3/18/22
'''
from qiskit import assemble, transpile, ClassicalRegister, QuantumRegister
from qiskit.visualization import array_to_latex, plot_histogram
from qiskit.tools.monitor import job_monitor 
'''


<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


'\nfrom qiskit import assemble, transpile, ClassicalRegister, QuantumRegister\nfrom qiskit.visualization import array_to_latex, plot_histogram\nfrom qiskit.tools.monitor import job_monitor \n'

## Load IBM-Q account

In [ ]:
#loading account
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')
print(f'provider: {provider}\n')
print(f'backends: {provider.backends()}')

provider: <AccountProvider for IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>

backends: [<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_armonk') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_santiago') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_bogota') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_casablanca') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-education', group='fermilab-1', project='qjs-for-hep')>, <IBMQBackend('ibmq_jakart

In [ ]:
#noise model for simulator (readout error)
'''
#noise model with readout error
nqubits = 3 
noise_model = noise.NoiseModel()
for qi in range(nqubits):
    read_err = noise.errors.readout_error.ReadoutError([[0.88, 0.12],[0.08,0.92]])
    noise_model.add_readout_error(read_err, [qi])
'''


#noise model based on real machine
device_backend = provider.get_backend('ibmq_belem')
noise_model = noise.NoiseModel.from_backend(device_backend)

backend = provider.get_backend('ibmq_qasm_simulator') #not sure how this is different from provider.backend.ibmq_qasm_simulator
print(type(backend.options))
backend.options.update_options(noise_model=noise_model)
print(f'backend options: {backend.options}') #can't figure out a way to set the backend options...

#backend2 = Aer.get_backend('qasm_simulator')
#print(f'backend 2 options: {backend2.options}') #can't figure out a way to set the backend options...

'''
#backend for running on real machine
backend = provider.get_backend('ibmq_belem')
program = provider.runtime.program('circuit-runner')
'''



<class 'qiskit.providers.options.Options'>
backend options: Options(shots=4000, memory=False, qubit_lo_freq=None, meas_lo_freq=None, schedule_los=None, meas_level=<MeasLevel.CLASSIFIED: 2>, meas_return=<MeasReturnType.AVERAGE: 'avg'>, memory_slots=None, memory_slot_size=100, rep_time=None, rep_delay=None, init_qubits=True, use_measure_esp=None, noise_model=<NoiseModel on ['cx', 'reset', 'measure', 'id', 'sx', 'x']>, seed_simulator=None)


"\n#backend for running on real machine\nbackend = provider.get_backend('ibmq_belem')\nprogram = provider.runtime.program('circuit-runner')\n"

In [ ]:
print(backend1.options.noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['cx', 'reset', 'measure', 'id', 'sx', 'x']
  Qubits with noise: [0, 1, 2, 3, 4]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2,)), ('id', (3,)), ('id', (4,)), ('sx', (0,)), ('sx', (1,)), ('sx', (2,)), ('sx', (3,)), ('sx', (4,)), ('x', (0,)), ('x', (1,)), ('x', (2,)), ('x', (3,)), ('x', (4,)), ('cx', (4, 3)), ('cx', (3, 4)), ('cx', (3, 1)), ('cx', (1, 3)), ('cx', (2, 1)), ('cx', (1, 2)), ('cx', (1, 0)), ('cx', (0, 1)), ('reset', (0,)), ('reset', (1,)), ('reset', (2,)), ('reset', (3,)), ('reset', (4,)), ('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,))]


In [ ]:
print(backend.options.noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['cx', 'reset', 'measure', 'id', 'sx', 'x']
  Qubits with noise: [0, 1, 2, 3, 4]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2,)), ('id', (3,)), ('id', (4,)), ('sx', (0,)), ('sx', (1,)), ('sx', (2,)), ('sx', (3,)), ('sx', (4,)), ('x', (0,)), ('x', (1,)), ('x', (2,)), ('x', (3,)), ('x', (4,)), ('cx', (4, 3)), ('cx', (3, 4)), ('cx', (3, 1)), ('cx', (1, 3)), ('cx', (2, 1)), ('cx', (1, 2)), ('cx', (1, 0)), ('cx', (0, 1)), ('reset', (0,)), ('reset', (1,)), ('reset', (2,)), ('reset', (3,)), ('reset', (4,)), ('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,))]


## Define functions

In [ ]:
def gauge_kinetic(epsilon):
    circuit=QuantumCircuit(1)
    circuit.rx(-epsilon/2,0)
    U_kg = circuit.to_gate()
    U_kg.name = "U$_{Kg}$"
    return U_kg

def fermion_mass(epsilon,mass,eta):
    circuit=QuantumCircuit(1)
    circuit.rz(-epsilon*mass * eta,0)
    U_m = circuit.to_gate()
    U_m.name = "U$_m$"
    return U_m

def fermion_hopping_opt2(epsilon,eta):
    circuit= QuantumCircuit(3)
    circuit.cx(0,2)
    circuit.h(0)
    circuit.cx(1,0)
    circuit.cx(0,2)
    circuit.rz(epsilon/4 * eta,0)
    circuit.rz(-epsilon/4 * eta,2)
    circuit.cx(0,2)
    circuit.cx(1,0)
    circuit.h(0)
    circuit.cx(0,2)
    U_fho2 = circuit.to_gate()
    U_fho2.name = "U$_{fho2}$"
    return U_fho2

#mean fermion number function for noisy counts 
def get_mean_fermion_number(counts):
    mean = 0
    values= list(counts.values())
    total_counts = sum(values)
    for s in counts:
        p = s[-1]
        if p == '1':
             mean = mean + (counts[s]/total_counts)
    return mean

#mean fermion number function for mitigated counts 
def get_mean_fermion_number2(counts):
    mean = 0
    for s in counts:
        p = s[-1]
        if p == '1':
            mean = mean + counts[s]
    return mean

#bootstrap error function for noisy counts
def get_bootstrap_error(counts):
    values= list(counts.values())
    nshots=sum(values)
    B = 100
    k = list(counts.keys())
    prob = [counts[a]/nshots for a in k]
    means = []
    for b in range(B):
        m = 0
        samples = np.random.choice(k, size=nshots, p=prob)
        for s in samples:
            p = s[-1]
            if p == '1':
                m = m + (1/nshots)
        means.append(m)
    return np.std(means), nshots

#bootstrap error function for mitigated counts 
def get_bootstrap_error2(counts, nshots):
    values= list(counts.values())
    B = 100
    k = list(counts.keys())
    means = []
    for b in range(B):
        m = 0
        samples = np.random.choice(k, size=nshots, p=values)
        for s in samples:
            p = s[-1]
            if p == '1':
                m = m + (1/nshots)
        means.append(m)
    return np.std(means)

## Run simulation

In [ ]:
#sim code clement sent (thanks clement)
nqubits = 3 #replaced 2*N-1 with nqubits everywhere RV 3/18/22
epsilon = 1
mass = 1.0

counts=[]
counts2=[]
noisymeans = []
mitigatedmeans = []
noisyerrs= []
mitigatederrs = []
Ts=[]

for T in range(int(0/epsilon),int(3/epsilon)):
    Ts.append(T)
    qc = QuantumCircuit(nqubits, nqubits)

    qc.x(0)
    qc.h(0)

    for t in range(T):
        for n in range(0,nqubits+1,2):
            qc.append(fermion_mass(epsilon,mass,(-1)**(n/2+1)),[n])
        for l in range(1,nqubits,2):
            qc.append(gauge_kinetic(epsilon),[l])
        for n in range(0,nqubits-2,2):
            qc.append(fermion_hopping_opt2(epsilon, (-1)**(n/2)),[n,n+1,n+2])
    
    #creating the circut and running it using qiskit runtime 
    qc.measure(range(nqubits), range(nqubits)) #changed 3 --> nqubits RV 3/18/22
    
    #if we don't specify 'shots' then the default 1024 will be used
    program_inputs = {
    'circuits': qc,
    'optimization_level': 3,
    'measurement_error_mitigation': True,
    'memory': True,
    'shots': 256
    }
    options = {'backend_name': backend.name()}
    job = provider.runtime.run(program_id="circuit-runner",
                               options=options,
                               inputs=program_inputs
                              )
    
    #appending noisy dictionary to counts
    result = job.result(decoder=RunnerResult)
    noisy = result.get_counts()
    counts.append(noisy)
    
    #appending mitigated diction to counts2 making sure to change the keys into states
    mitigated = result.get_quasiprobabilities().nearest_probability_distribution()
    dict2 = {}
    for key1 in mitigated:
        key2 = (bin(key1)[2:]).zfill(nqubits)
        dict2[key2] = mitigated[key1]
    counts2.append(dict2)
    
    #getting errs + mean fermion number 
    noisymeans.append(get_mean_fermion_number(counts[T]))  
    mitigatedmeans.append(get_mean_fermion_number2(counts2[T]))
    
    #getting bootstrap error for both noisy and mitigated results 
    #the get bootstrap error function returns nshots as well so that the second getbootstraperror function can take it in as an argument and doesn't have to calculate it 
    bootstrap_error = get_bootstrap_error(counts[T])
    noisyerrs.append(bootstrap_error[0])
    nshots = bootstrap_error[1]
    mitigatederrs.append(get_bootstrap_error2(counts2[T], nshots))  

In [ ]:
result_dict = result.to_dict()

#for key in result_dict:
#    print(f'{key}: {result_dict[key]}\n')
print(result_dict.keys(),'\n')
print(result_dict['results'][0].keys(),'\n')
print(f"*** result_dict['results'][0]['metadata']['noise']: {result_dict['results'][0]['metadata']['noise']} ***\n")

for key in result_dict['results'][0]:
    res = result_dict['results'][0][key]
    print(f"{key}: {res}\n")

dict_keys(['backend_name', 'backend_version', 'qobj_id', 'job_id', 'success', 'results', 'date', 'status', 'header', 'metadata', 'time_taken']) 

dict_keys(['shots', 'success', 'data', 'meas_level', 'header', 'status', 'seed_simulator', 'metadata', 'time_taken']) 

*** result_dict['results'][0]['metadata']['noise']: ideal ***

shots: 256

success: True

data: {'counts': {'0x3': 29, '0x4': 23, '0x1': 69, '0x6': 2, '0x2': 29, '0x0': 104}, 'memory': ['0x2', '0x0', '0x2', '0x1', '0x2', '0x1', '0x1', '0x1', '0x0', '0x1', '0x0', '0x0', '0x4', '0x1', '0x0', '0x2', '0x1', '0x3', '0x0', '0x3', '0x1', '0x0', '0x1', '0x1', '0x2', '0x1', '0x0', '0x1', '0x0', '0x1', '0x0', '0x0', '0x4', '0x3', '0x2', '0x2', '0x4', '0x0', '0x2', '0x0', '0x1', '0x0', '0x4', '0x0', '0x0', '0x2', '0x0', '0x1', '0x2', '0x2', '0x1', '0x4', '0x0', '0x3', '0x0', '0x1', '0x1', '0x3', '0x0', '0x0', '0x1', '0x0', '0x1', '0x3', '0x2', '0x0', '0x0', '0x0', '0x0', '0x0', '0x2', '0x1', '0x4', '0x1', '0x1', '0x1', '0x2', '0x2', '0

## Print data (_need to add saving to file!_)

In [ ]:
print(f'counts: {counts}')
print(f'percentages: {counts2}')

print('\nmean fermion number (noisy)')
for i, nf in enumerate(noisymeans):
    print(f'T={i}: nf = {round(nf,3)} +/- {round(noisyerrs[i],3)}')
    
print('\nmean fermion number (corrected for readout error)')
for i, nf in enumerate(mitigatedmeans):
    print(f'T={i}: nf = {round(nf,3)} +/- {round(mitigatederrs[i],3)}')

counts: [{'000': 124, '001': 132}, {'011': 5, '100': 8, '001': 121, '010': 1, '110': 1, '000': 120}, {'011': 29, '100': 23, '001': 69, '110': 2, '010': 29, '000': 104}]
percentages: [{'000': 0.484375, '001': 0.515625}, {'010': 0.00390625, '110': 0.00390625, '011': 0.01953125, '100': 0.03125, '000': 0.46875, '001': 0.47265625}, {'110': 0.0078125, '100': 0.08984375, '010': 0.11328125, '011': 0.11328125, '001': 0.26953125, '000': 0.40625}]

mean fermion number (noisy)
T=0: nf = 0.516 +/- 0.03
T=1: nf = 0.492 +/- 0.032
T=2: nf = 0.383 +/- 0.03

mean fermion number (corrected for readout error)
T=0: nf = 0.516 +/- 0.029
T=1: nf = 0.492 +/- 0.029
T=2: nf = 0.383 +/- 0.03


## Plot results

In [ ]:
plt.errorbar(Ts, noisymeans , yerr=noisyerrs, ls='', marker='o', color='b')
plt.errorbar(Ts, mitigatedmeans , yerr=mitigatederrs, ls='', marker='o', color='r')

In [ ]:
X = ['0','1','2','3', '4', '5']
X_axis = np.arange(len(X))

plt.bar(X_axis - 0.2, noisymeans, 0.4, label = 'noisy')
plt.bar(X_axis + 0.2, mitigatedmeans, 0.4, label = 'mitigated')

  
plt.xticks(X_axis, X)
plt.xlabel("Timestep")
plt.ylabel("Mean Fermion Number")
plt.title("Mean Fermion Number: mitigated vs noisy results")
plt.ylim([0.2, 0.9])
plt.legend()
plt.show()

In [ ]:
X = ['0','1','2','3', '4', '5']
X_axis = np.arange(len(X))

plt.bar(X_axis - 0.2, noisyerrs, 0.4, label = 'noisy')
plt.bar(X_axis + 0.2, mitigatederrs, 0.4, label = 'mitigated')

  
plt.xticks(X_axis, X)
plt.xlabel("Timestep")
plt.ylabel("Errors")
plt.title("Bootstrap Errors: migigated vs noisy results")
plt.ylim([0.01, 0.02])
plt.legend()
plt.show()